In [2]:
from sklearn.datasets import fetch_openml


In [3]:
fetch_openml?

Signature:
fetch_openml(
    name: Optional[str] = None,
    *,
    version: Union[str, int] = 'active',
    data_id: Optional[int] = None,
    data_home: Union[str, os.PathLike, NoneType] = None,
    target_column: Union[str, List, NoneType] = 'default-target',
    cache: bool = True,
    return_X_y: bool = False,
    as_frame: Union[str, bool] = 'auto',
    n_retries: int = 3,
    delay: float = 1.0,
    parser: str = 'warn',
    read_csv_kwargs: Optional[Dict] = None,
)
Docstring:
Fetch dataset from openml by name or dataset id.

Datasets are uniquely identified by either an integer ID or by a
combination of name and version (i.e. there might be multiple
versions of the 'iris' dataset). Please give either name or data_id
(not both). In case a name is given, a version can also be
provided.

Read more in the :ref:`User Guide <openml>`.

.. versionadded:: 0.20

.. note:: EXPERIMENTAL

    The API is experimental (particularly the return value structure),
    and might have small backwa

In [7]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [10]:
X, y = fetch_openml(data_id=554,parser='pandas', return_X_y=True)

D:\STUDY\MachineLearning\ml_classification_project\venv\Lib\site-packages\sklearn\datasets\_openml.py:110: UserWarning: A network error occurred while downloading https://api.openml.org/api/v1/json/data/554. Retrying...
  warn(


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>

In [11]:
import sklearn
sklearn.__version__

'1.3.2'